In [1]:
'''
2022.3.9
术后预后模型：
    生存时间模型
    风险评分模型
'''

'\n2022.3.9\n术后预后模型：\n    生存时间模型\n    风险评分模型\n'

In [2]:
import pandas as pd
import numpy as np

import sys
import re
import os
project_path = os.getcwd()

# 建模

## 读入数据

In [3]:
df_model =pd.read_excel(project_path +'/data/result/feature_engineering/df_4.5_model_data_forward_after_accuracy.xlsx')
if 'Unnamed: 0' in df_model.columns:
    df_model = df_model.drop(['Unnamed: 0'], axis=1)

In [4]:
df_model.columns

Index(['label', 'biliary_disease', 'AJCC_8', 'MSKCC', 'gender', 'smoke',
       'drinking', 'blood_type', 'HBcAb', 'DB', 'emaciation', 'tumor_AFP',
       'tumor_CEA', 'tumor_CA199', 'tumor_CA125', 'tumor_size', 'HBsAg', 'LC',
       'PTCD_ERCP', 'surgery_bleeding', 'surgery_plasm', 'surgery_result'],
      dtype='object')

In [5]:
df_model.shape

(417, 22)

In [6]:
df_model['label'].value_counts()

2    129
1    111
0    100
3     77
Name: label, dtype: int64

In [7]:
discrete_col=['biliary_disease','AJCC_8', 'MSKCC', 'gender', 'smoke','drinking', 'blood_type', 'HBcAb',
             'emaciation','HBsAg', 'LC','PTCD_ERCP','surgery_result']

continuous_col=[x for x in df_model.columns if x not in discrete_col]
continuous_col.remove('label')

## 数据归一化

In [8]:
# 防止不同维特征数据差距过大，影响建模效果
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
# df_model[continuous_col]=ss.fit_transform(df_model[continuous_col])
for i in continuous_col:
    df_model[[i]] = ss.fit_transform(df_model[[i]])

In [9]:
df_model.columns

Index(['label', 'biliary_disease', 'AJCC_8', 'MSKCC', 'gender', 'smoke',
       'drinking', 'blood_type', 'HBcAb', 'DB', 'emaciation', 'tumor_AFP',
       'tumor_CEA', 'tumor_CA199', 'tumor_CA125', 'tumor_size', 'HBsAg', 'LC',
       'PTCD_ERCP', 'surgery_bleeding', 'surgery_plasm', 'surgery_result'],
      dtype='object')

In [10]:
df_model.to_excel(project_path+'/data/result/modeling/df_1.2_数据归一化.xlsx')

## 插补数据

In [11]:
# 使用随机森林对缺失值进行插补
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
def missing_value_interpolation(df):
    df = df.reset_index(drop=True)
    # 提取存在缺失值的列名
    missing_list = []
    for i in df.columns:
        if df[i].isnull().sum()>0:
            missing_list.append(i)
    missing_list_copy = missing_list.copy()
    # 用该列未缺失的值训练随机森林，然后用训练好的rf预测缺失值
    for i in range(len(missing_list)):
        name=missing_list[0]
        df_missing = df[missing_list_copy]
        # 将其他列的缺失值用0表示。
        missing_list.remove(name)
        for j in missing_list:
            df_missing[j]=df_missing[j].astype('str').apply(lambda x: 0 if x=='nan' else x)
        df_missing_is = df_missing[df_missing[name].isnull()]
        df_missing_not = df_missing[df_missing[name].notnull()]
        y = df_missing_not[name]
        x = df_missing_not.drop([name],axis=1)

        rfr = RandomForestRegressor(n_estimators=300,
                                    random_state=3)
        rfr.fit(x, y)
        #预测缺失值
        predict = rfr.predict(df_missing_is.drop([name],axis=1))
        #填补缺失值
        df.loc[df[name].isnull(),name] = predict
    return df

In [12]:
# 插补建模数据
df_model_cb=missing_value_interpolation(df_model)
# df_model_cb=df_model

KeyboardInterrupt: 

In [ ]:
df_model_cb.shape

In [ ]:
# 保存插补数据
df_model_cb.to_excel(project_path + '/data/result/modeling/df_1.3_model_data_插补.xlsx')

## 划分数据集

### 计算随机数种子

In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score,average_precision_score,precision_recall_curve
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,accuracy_score

# 划分训练集和测试集，比例为8:2
x = df_model_cb.drop(['label'],axis=1)
y = df_model_cb['label']

seeds_list=[]
cat_f1_list=[]
for i in range(101):
    
    tran_x, test_x, tran_y, test_y = train_test_split(x, y, test_size=0.2, random_state=i)
    
    # 分类数据过采样
    from imblearn.over_sampling import SMOTE,ADASYN 
    sm = SMOTE(random_state=0)
    tran_x_sm,tran_y_sm = sm.fit_resample(tran_x,tran_y)
#     tran_x_sm,tran_y_sm=tran_x,tran_y
    
    import xgboost
    cat_model=xgboost.XGBClassifier(max_depth=5,
                        learning_rate=0.01,
                        n_estimators=500,
                        min_child_weight=0.5,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=10,
                        subsample=0.5,
                        colsample_bytree=0.8,
                        nthread=4,
                        scale_pos_weight=1,
                        random_state=3)
    # 分类模型
    cat_model.fit(tran_x_sm,tran_y_sm)
    cat_predictions=cat_model.predict(test_x)
    cat_f1=f1_score(test_y,cat_predictions,average='macro')
    # 防止分类数据的测试集划分不平衡
    if not (2 >=(test_y.value_counts().values[0])/(test_y.value_counts().values[-1]) >=1):
        continue

#     import catboost
#     # CatBoost模型
#     cat_model=catboost.CatBoostRegressor(iterations=300, 
#                                           learning_rate=0.2, 
#                                           depth=6,
#                                           l2_leaf_reg=2,
#                                           subsample=1,
#                                           loss_function='RMSE', # 'CrossEntropy',
#                                           random_state=3)
#     # 回归模型
#     cat_model.fit(tran_x,tran_y)
#     cat_predictions=cat_model.predict(test_x)
#     cat_f1=r2_score(test_y,cat_predictions)
    
    seeds_list.append(i)
    cat_f1_list.append(cat_f1)

In [ ]:
test_y.value_counts()

In [ ]:
df_seeds=pd.DataFrame(data={'seed':seeds_list,
                           'cat_f1':cat_f1_list})
df_seeds=df_seeds.sort_values(['cat_f1'], ascending=0).reset_index(drop=True)
df_seeds.to_excel(project_path+'/data/df_seeds.xlsx')

In [ ]:
df_seeds.loc[0,'seed']

### 划分数据集

In [ ]:
# 分类随机数种子
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
# 划分训练集和测试集，比例为8:2
x = df_model_cb.drop(['label'],axis=1)
y = df_model_cb['label']

seed_index=df_seeds.loc[0,'seed']
tran_x, test_x, tran_y, test_y = train_test_split(x, y, test_size=0.2, random_state=seed_index)

In [ ]:
seed_index

In [ ]:
df_model.shape

In [ ]:
print(tran_x.shape)
print(test_x.shape)

In [ ]:
df_model.label.value_counts()

In [ ]:
tran_y.value_counts()

In [ ]:
test_y.value_counts()

In [ ]:
tran_x.head()

In [ ]:
test_x.head()

In [ ]:
test_y

In [ ]:
type(test_y)

## 训练集过采样

In [ ]:
# 进行过采样
from imblearn.over_sampling import SMOTE,ADASYN 
from imblearn.combine import SMOTETomek
sm = SMOTE(random_state=0)
# sm=ADASYN(random_state=0)

tran_x_sm,tran_y_sm = sm.fit_resample(tran_x,tran_y)

In [ ]:
tran_x_sm.shape

In [ ]:
tran_y_sm.value_counts()

In [ ]:
tran_x_sm.head()

In [ ]:
tran_x_sm.to_excel(project_path+'/data/tran_x_sm.xlsx')
tran_y_sm.to_excel(project_path+'/data/tran_y_sm.xlsx')

## 多分类模型：5-fold cross validation

In [ ]:
seed_index

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold 

# 划分训练集和测试集，比例为8:2
x = df_model_cb.drop(['label'],axis=1)
y = df_model_cb['label']
# 五折交叉验证
kf = KFold(n_splits=5,random_state=seed_index,shuffle=True)

df_auc_accuracy=pd.DataFrame()
df_precision_f1=pd.DataFrame()
temp_importance_list=[]

for train_index, test_index in kf.split(x):

    tran_x,test_x,tran_y,test_y=x.values[train_index],x.values[test_index],y.values[train_index],y.values[test_index]

    tran_x_sm,tran_y_sm = sm.fit_resample(tran_x,tran_y)
    
    from sklearn.metrics import r2_score,average_precision_score,precision_recall_curve,brier_score_loss
    from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,accuracy_score
    import xgboost
    # XGBoost模型
    xgb_model=xgboost.XGBClassifier(max_depth=3,
                            learning_rate=0.01,
                            n_estimators=500,
                            min_child_weight=0.3,
                            eta=0.1,
                            gamma=0.4,
                            reg_lambda=10,
                            subsample=0.8,
                            colsample_bytree=0.8,
                            nthread=4,
                            scale_pos_weight=1,
                            random_state=3)
    xgb_model.fit(tran_x_sm,tran_y_sm)
    xgb_predictions=xgb_model.predict(test_x)


    import lightgbm
    # LightGBM模型
    lgbm_model=lightgbm.LGBMClassifier(iterations=300, 
                                      max_depth=4,
                                      min_child_weight=0.5,
                                      gamma=0.5,
                                       reg_lambda=5,
                                      subsample=0.8,
                                      learning_rate=0.02, 
                                      loss_function='CrossEntropy',
                                      random_state=3)
    lgbm_model.fit(tran_x_sm,tran_y_sm)
    lgbm_predictions=lgbm_model.predict(test_x)


    import catboost
    # CatBoost模型
    cat_model=catboost.CatBoostClassifier(iterations=400, 
                                          learning_rate=0.09, 
                                          depth=3,
                                          l2_leaf_reg=2,
                                          loss_function='MultiClass',
                                          random_state=3)
    cat_model.fit(tran_x_sm,tran_y_sm)
    cat_predictions=cat_model.predict(test_x)
    
    # 随机森林
    from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
    from sklearn.model_selection import GridSearchCV
    # 列出参数列表
    tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
    # 进行参数的搜索组合
    grid = GridSearchCV(RandomForestClassifier(), param_grid=tree_grid_parameter, cv=3)
    # 根据已有数据去拟合随机森林模型
    grid.fit(tran_x_sm, tran_y_sm)
    rf_model = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'],
                                max_depth=8,
                                random_state=3)
    rf_model.fit(tran_x_sm, tran_y_sm)
    # 预测缺失值
    rf_predictions = rf_model.predict(test_x)


    # GBDT
    # 列出参数列表
    gbdt_model = GradientBoostingClassifier(n_estimators=300,
                                learning_rate=0.1,
                                max_depth=3,
                                subsample=0.5,
                                random_state=3)
    gbdt_model.fit(tran_x_sm,tran_y_sm)
    # 预测缺失值
    gbdt_predictions = gbdt_model.predict(test_x)


    # SVR
    from sklearn.svm import SVR,SVC
    # 回归模型
    # svr = SVR(kernel='linear', C=1.25)
    # 分类模型
    svr_model = SVC(kernel='rbf',
                    C=50,
                    cache_size=200,
                    probability=True,
                    random_state=3)
    svr_model.fit(tran_x_sm,tran_y_sm)
    svr_predictions=svr_model.predict(test_x)


    # Linear回归，Lasso回归，领回归，logistic回归
    from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet,LogisticRegression
    lcv_model = LogisticRegression(penalty='l2',
                             C=5,
                            solver='lbfgs',
                             max_iter=100,
                            random_state=3)
    # lcv = Lasso()
    # lcv = Ridge()
    lcv_model.fit(tran_x_sm, tran_y_sm)
    lcv_predictions = lcv_model.predict(test_x)

    # ANN
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import classification_report,confusion_matrix

    ANN_model = MLPClassifier(alpha=0.1, 
                        hidden_layer_sizes=[100,], 
                        solver='adam', 
                        activation='relu', 
                        random_state=3)
    ANN_model.fit(tran_x_sm, tran_y_sm)
    ANN_predictions=ANN_model.predict(test_x)

    # TabNet
    import torch
    from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
    from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
    TabNet_model = TabNetMultiTaskClassifier(
                            n_d=8,
                            n_a=8,
                            n_steps=5,
                            gamma=1,
#                            optimizer_fn=torch.optim.Adam,
#                            optimizer_params=dict(lr=2e-2),
#                            scheduler_params={"step_size":50, # how to use learning rate scheduler
#                                              "gamma":0.9},
#                            scheduler_fn=torch.optim.lr_scheduler.StepLR,
                           mask_type='entmax') # "sparsemax"
    tran_x_x, tran_x_valid, tran_y_y, tran_y_valid = train_test_split(tran_x_sm, tran_y_sm, test_size=0.125, random_state=3)

    TabNet_model.fit(X_train=tran_x_sm, 
            y_train=tran_y_sm.reshape(-1,1),
            max_epochs=200, 
            patience=50,
            batch_size=64, 
            virtual_batch_size=16,
            num_workers=0,
            drop_last=False,
            loss_fn=[torch.nn.functional.cross_entropy]) # Optional, just an example of list usage
    TabNet_predictions=TabNet_model.predict(test_x)

    # 计算评价指标compute evaluation metrics
    from sklearn.metrics import classification_report,confusion_matrix
    # 统一模型输出结果
    df_model_result=pd.DataFrame(
        columns=['model','index','precision','recall','f1-score','support','accuracy','AUC','sensitivity','specificity'])

    model_list=[xgb_model,lgbm_model,cat_model,rf_model,gbdt_model,svr_model,lcv_model,ANN_model,TabNet_model]
    model_name_list=['XGBoost','LGBM','CatBoost','RF','GBDT','SVR','LR','ANN','TabNet']
#     model_list=[TabNet_model]
#     model_name_list=['TabNet']
    
    temp_auc=pd.DataFrame()    
    for model,name in zip(model_list,model_name_list):
        print(name)
        # 计算accuracy和AUC
        # tabnet predict_proba结果是三维数组，无法计算auc，需要reshape(-1,6),所有行 x 6列
        test_y_score=np.reshape(model.predict_proba(test_x),(-1,4))
        auc=roc_auc_score(test_y,test_y_score,multi_class='ovr')
        auc=round(auc,2)
        # tabnet predict结果是三维数组，无法计算auc，需要reshape
        predictions=np.reshape(model.predict(test_x),(-1,1)).astype(str)
        accuracy=accuracy_score(test_y.astype(str),predictions)
        accuracy=round(accuracy,2)
        # 计算precision、recall、F1
        precision=precision_score(test_y.astype(str),predictions,average='macro')
        precision=round(precision,2)
        recall=recall_score(test_y.astype(str),predictions,average='macro')
        recall=round(recall,2)
        f1=f1_score(test_y.astype(str),predictions,average='macro')
        f1=round(f1,2)
        temp_auc.loc[temp_auc.shape[0],['model','precision','recall','f1','accuracy','AUC']]=\
                                                                    [name,precision,recall,f1,accuracy,auc]
    df_auc_accuracy=pd.concat([df_auc_accuracy,temp_auc],axis=0)
                                       
    # 变量重要性评分
    importance = xgb_model.feature_importances_
    df_importance_temp=pd.DataFrame(data={'特征':x.columns,'重要性评分':importance})
    temp_importance_list.append(df_importance_temp)

In [ ]:
df_model_result=df_auc_accuracy.groupby(['model'])[['precision','recall','f1','accuracy','AUC']].mean().reset_index()
df_model_result=df_model_result.round(2)

In [ ]:
df_model_result

In [ ]:
# 排序
df_model_result['temp_num']=[8,3,5,2,6,4,7,9,1]
df_model_result=df_model_result.sort_values(by=['temp_num'],ascending=True).drop(['temp_num'],axis=1)

In [ ]:
df_model_result

In [ ]:
# 保存模型测试效果
df_model_result.to_excel(project_path+'/data/result/modeling/df_1.6_分类_模型测试效果_5折交叉验证_after.xlsx')

In [ ]:
df_importance=pd.concat(temp_importance_list,axis=0)
df_importance=df_importance.groupby(['特征'])[['重要性评分']].mean().sort_values(['重要性评分'],ascending=False).reset_index()
df_importance=df_importance.round(3)

In [ ]:
df_importance

In [ ]:
df_importance.to_excel(project_path+'/data/result/modeling/df_1.6_模型重要性评分_after.xlsx')

## 术后风险评分数据集

### 构建术后生存分析数据集

In [526]:
df_importance =pd.read_excel(project_path +'/data/result/modeling/df_1.6_模型重要性评分_after.xlsx')
if 'Unnamed: 0' in df_importance.columns:
    df_importance = df_importance.drop(['Unnamed: 0'], axis=1)

In [527]:
importance_col=df_importance['特征'].to_list()

In [528]:
importance_col

['surgery_result',
 'AJCC_8',
 'blood_type',
 'tumor_CA19-9',
 'MSKCC',
 'tumor_CA125',
 'HBcAb',
 'surgery_plasm',
 'HBsAg',
 'surgery_bleeding',
 'tumor_size',
 'smoke',
 'emaciation',
 'drinking',
 'DB',
 'tumor_AFP',
 'tumor_CEA',
 'PTCD_ERCP',
 'biliary_disease',
 'gender',
 'LC']

In [529]:
type(importance_col)

list

In [530]:
# 加入生存期和标签
survival_col=['target_3Y_death','target_survival_month']
importance_col.extend(survival_col)

In [531]:
importance_col

['surgery_result',
 'AJCC_8',
 'blood_type',
 'tumor_CA19-9',
 'MSKCC',
 'tumor_CA125',
 'HBcAb',
 'surgery_plasm',
 'HBsAg',
 'surgery_bleeding',
 'tumor_size',
 'smoke',
 'emaciation',
 'drinking',
 'DB',
 'tumor_AFP',
 'tumor_CEA',
 'PTCD_ERCP',
 'biliary_disease',
 'gender',
 'LC',
 'target_3Y_death',
 'target_survival_month']

In [576]:
df_dataset_cox_after =pd.read_excel(project_path +'/data/processed_data/df_3.1.1_术后预后cox数据集.xlsx')
if 'Unnamed: 0' in df_dataset_cox_after.columns:
    df_dataset_cox_after = df_dataset_cox_after.drop(['Unnamed: 0'], axis=1)

In [533]:
df_dataset_cox_after.shape

(451, 75)

In [534]:
df_dataset_cox_after_importance=df_dataset_cox_after[importance_col]

In [535]:
df_dataset_cox_after_importance.shape

(451, 23)

In [536]:
df_dataset_cox_after_importance.columns

Index(['surgery_result', 'AJCC_8', 'blood_type', 'tumor_CA19-9', 'MSKCC',
       'tumor_CA125', 'HBcAb', 'surgery_plasm', 'HBsAg', 'surgery_bleeding',
       'tumor_size', 'smoke', 'emaciation', 'drinking', 'DB', 'tumor_AFP',
       'tumor_CEA', 'PTCD_ERCP', 'biliary_disease', 'gender', 'LC',
       'target_3Y_death', 'target_survival_month'],
      dtype='object')

### 训练集与测试集

In [537]:
discrete_col=['surgery_result', 'AJCC_8', 'blood_type','MSKCC','HBcAb', 'surgery_plasm', 'HBsAg', 'surgery_bleeding','smoke', 
              'emaciation', 'drinking','PTCD_ERCP', 'biliary_disease', 'gender', 'LC', 'target_3Y_death']
continuous_col=[x for x in df_dataset_cox_after_importance.columns if x not in discrete_col]

#### 筛选随机数种子

In [538]:
from scipy.stats import kstest,shapiro
##检验是否正态
def norm_test(data):
    if len(data) > 30:
        norm, p = kstest(data, 'norm')
    else:
        norm, p = shapiro(data)
    #print(t,p)
    if p>=0.05:
        return True
    else:
        return False

In [539]:
import scipy.stats as st
# 连续变量的显著性检验
def test2(data_b, data_p):
    if norm_test(data_b) and norm_test(data_p):
        x = 1
        y = '独立样本T检验'
        t, p = st.ttest_ind(list(data_b),list(data_p), nan_policy='omit')
    else:
        x = 0
        y = 'Mann-Whitney U检验'
        t,p = st.mannwhitneyu(list(data_b),list(data_p))
    return x,y,t,p

In [540]:
def sig_test(df_high,df_low,list1):

    feature_list=[]  # 特征列表
    y_list=[]  # 显著性检验方法
    t_list=[]  # 统计量
    p_list=[]  # p值
    result_list=[]  # 是否显著
    high_mean_list=[]
    low_mean_list=[]

    for i in list1:
        print(i)
        # 高剂量组统计
        df_high_nt=df_high[df_high[i].notnull()]
        data_high=df_high_nt[i]
        high_mean=round(data_high.mean(),2)
        
        df_low_nt=df_low[df_low[i].notnull()]
        data_low=df_low_nt[i]
        low_mean=round(data_low.mean(),2)

        # 计算高低剂量组显著性差异
        if data_high.shape[0] >= 10 and data_low.shape[0]>=10:
            # 连续变量检验
            x,y,t,p = test2(data_high, data_low)
            t=round(t,2)
            p=round(p,3)
            if p <=0.05:
                sig='显著'
            else:
                sig='不显著'
            # 显著性 
            feature_list.append(i)
            y_list.append(y)
            t_list.append(t)
            p_list.append(p)
            result_list.append(sig)
            high_mean_list.append(high_mean)
            low_mean_list.append(low_mean)

    df_result=pd.DataFrame({'特征':feature_list,
                            '高剂量均值':high_mean_list,
                            '低剂量均值':low_mean_list,
                            '检验指标':y_list,
                            't值':t_list,
                            'p值':p_list,
                            '显著性结果':result_list})
    return df_result

In [541]:
# 筛选出划分效果最均衡的随机数种子
from sklearn.model_selection import train_test_split
# 划分训练集和测试集，比例为8:2
seed_index_list=[]
p_mean_list=[]
for i in range(100):
    df_after_tran, df_after_test = train_test_split(df_dataset_cox_after_importance, test_size=0.2, random_state=i)
    df_continuous_sig = sig_test(df_after_tran,df_after_test,continuous_col)
    p_mean=df_continuous_sig['p值'].mean() + df_continuous_sig['p值'].std()
    seed_index_list.append(i)
    p_mean_list.append(p_mean)
    
df_p_mean=pd.DataFrame(data={'seed_index':seed_index_list,
                            'p_mean':p_mean_list})
df_p_mean=df_p_mean.sort_values(['p_mean'],ascending=False).reset_index(drop=True)

tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month
tumor_CA19-9
tumor_CA125
tum

In [542]:
df_p_mean

,seed_index,p_mean
0,75,0.508878
1,96,0.502905
2,41,0.489131
3,42,0.487529
4,25,0.486739
...,...,...
95,94,0.274103
96,37,0.271496
97,3,0.247287
98,17,0.235961


In [543]:
seed_index=df_p_mean.loc[0,'seed_index']

In [544]:
seed_index

75

#### 数据集划分

In [545]:
from sklearn.model_selection import train_test_split
# 划分训练集和测试集，比例为8:2
df_after_ML_tran, df_after_ML_test = train_test_split(df_dataset_cox_after_importance, test_size=0.2,
                                                                random_state=seed_index)

In [546]:
df_after_ML_tran.to_excel(project_path+'/data/result/cox/df_1.7.2.2_after_ML_tran.xlsx')
df_after_ML_test.to_excel(project_path+'/data/result/cox/df_1.7.2.2_after_ML_test.xlsx')

#### 数据集同分布检验

In [547]:
discrete_col

['surgery_result',
 'AJCC_8',
 'blood_type',
 'MSKCC',
 'HBcAb',
 'surgery_plasm',
 'HBsAg',
 'surgery_bleeding',
 'smoke',
 'emaciation',
 'drinking',
 'PTCD_ERCP',
 'biliary_disease',
 'gender',
 'LC',
 'target_3Y_death']

In [548]:
# 判断训练集与测试集是否独立同分布
# 分类变量显著性分析
from scipy.stats import chi2_contingency
feature_list=[]
y_list=[]
t_list=[]
p_list=[]
sig_list=[]
for i in discrete_col:
    print(i)
    if len(df_after_ML_tran[i].value_counts().to_list()) == len(df_after_ML_test[i].value_counts().to_list()):
        result = chi2_contingency([df_after_ML_tran[i].value_counts().to_list(),
                                   df_after_ML_test[i].value_counts().to_list()])
        t,p=result[0:2]
        t=round(t,2)
        p=round(p,3)
    else:
        t=np.nan
        p=np.nan
    feature_list.append(i)
    y_list.append('卡方检验')
    t_list.append(t)
    p_list.append(p)
    if p <=0.05:
        sig='显著'
    else:
        sig='不显著'
    sig_list.append(sig)

surgery_result
AJCC_8
blood_type
MSKCC
HBcAb
surgery_plasm
HBsAg
surgery_bleeding
smoke
emaciation
drinking
PTCD_ERCP
biliary_disease
gender
LC
target_3Y_death


In [549]:
df_discrete_sig=pd.DataFrame(data={'feature':discrete_col,
                                        'method':y_list,
                                        't':t_list,
                                        'p':p_list,
                                        'result':sig_list})

In [550]:
df_discrete_sig

,feature,method,t,p,result
0,surgery_result,卡方检验,0.02,0.999,不显著
1,AJCC_8,卡方检验,NaN,NaN,不显著
2,blood_type,卡方检验,1.15,0.765,不显著
3,MSKCC,卡方检验,0.05,0.974,不显著
4,HBcAb,卡方检验,0.09,0.769,不显著
5,surgery_plasm,卡方检验,4.38,0.735,不显著
6,HBsAg,卡方检验,0.06,0.811,不显著
7,surgery_bleeding,卡方检验,NaN,NaN,不显著
8,smoke,卡方检验,0.00,0.982,不显著
9,emaciation,卡方检验,1.64,0.441,不显著


In [551]:
df_discrete_sig.to_excel(project_path+'/data/result/cox/df_1.7.2.3_after_分类变量数据同分布检验.xlsx')

In [552]:
# 连续变量显著性分析
df_continuous_sig = sig_test(df_after_ML_tran,df_after_ML_test,continuous_col)

tumor_CA19-9
tumor_CA125
tumor_size
DB
tumor_AFP
tumor_CEA
target_survival_month


In [553]:
df_continuous_sig

,特征,高剂量均值,低剂量均值,检验指标,t值,p值,显著性结果
0,tumor_CA19-9,375.22,376.99,Mann-Whitney U检验,15882.0,0.466,不显著
1,tumor_CA125,31.59,49.11,Mann-Whitney U检验,7058.0,0.465,不显著
2,tumor_size,2.88,2.93,Mann-Whitney U检验,9141.0,0.410,不显著
3,DB,147.15,135.15,Mann-Whitney U检验,11608.5,0.130,不显著
4,tumor_AFP,8.35,3.61,Mann-Whitney U检验,14773.0,0.444,不显著
5,tumor_CEA,6.33,5.90,Mann-Whitney U检验,15320.0,0.438,不显著
6,target_survival_month,24.98,20.10,Mann-Whitney U检验,8575.5,0.135,不显著


In [554]:
df_continuous_sig.to_excel(project_path+'/data/result/cox/df_1.7.2.3_after_连续变量数据同分布检验.xlsx')

#### 数据统计分布

In [555]:
# 分类变量数据统计
def stats_discrete(df_model,discrete_col):
    # 求分类变量比例
    df_discrete_stat=pd.DataFrame(columns=['变量名称','所有事件(%d)' % df_model.shape[0],'缺失率(%)'])
    for i in discrete_col:
        print(i)
        # 缺失率
        if df_model[i].isnull().sum()==0:
            miss_rate='0%'
        else:
            miss_rate=df_model[i].isnull().sum()/df_model.shape[0]
            miss_rate="%.2f%%" % (miss_rate * 100)      # 百分数输出
        df_discrete_stat.loc[df_discrete_stat.shape[0],['变量名称','缺失率(%)']]=[i+'，n(%)',miss_rate]

        # 分类变量单独统计
        name_list=[]
        num_perc_list=[]
        df_model_stat=df_model[df_model[i].notnull()].sort_values([i],ascending=True)

        # 二分类还是多分类变量
        if df_model_stat[i].nunique() <=2:
            if re.match('gender|性别',i):
                name_list=['男','女']
            else:
                name_list=['是','否']
            for name,value in zip(name_list,[1,0]):
                print(name)
                num=df_model_stat[df_model_stat[i]==value].shape[0]
                percent=num/df_model.shape[0]
                percent="%.2f%%" % (percent * 100)
                num_percent=str(num)+'('+percent+')'
                num_perc_list.append(num_percent)
        else:
            for value in sorted(df_model_stat[i].unique()):
                print(value)
                name_list.append(value)
                num=df_model_stat[df_model_stat[i]==value].shape[0]
                percent=num/df_model.shape[0]
                percent="%.2f%%" % (percent * 100)
                num_percent=str(num)+'('+percent+')'
                num_perc_list.append(num_percent)


        df_temp = pd.DataFrame(data={'变量名称':name_list,
                                     '所有事件(%d)' % df_model.shape[0]:num_perc_list})

        df_discrete_stat=pd.concat([df_discrete_stat,df_temp],axis=0)
        df_discrete_stat=df_discrete_stat.reset_index(drop=True)
    return df_discrete_stat

In [556]:
df_discrete_stat_tran=stats_discrete(df_after_ML_tran,discrete_col)
df_discrete_stat_tran.to_excel(project_path+'/data/result/cox/df_1.7.2.4_after_分类_tran_数据统计.xlsx')
df_discrete_stat_test=stats_discrete(df_after_ML_test,discrete_col)
df_discrete_stat_test.to_excel(project_path+'/data/result/cox/df_1.7.2.4_after_分类_test_数据统计.xlsx')

surgery_result
1.0
2.0
3.0
4.0
AJCC_8
0.0
2.0
3.0
4.0
blood_type
0.0
1.0
2.0
3.0
MSKCC
1.0
2.0
3.0
HBcAb
是
否
surgery_plasm
0.0
200.0
300.0
400.0
600.0
800.0
1000.0
1400.0
HBsAg
是
否
surgery_bleeding
5.0
10.0
50.0
80.0
100.0
150.0
200.0
250.0
300.0
350.0
400.0
500.0
600.0
700.0
750.0
800.0
1000.0
1200.0
1300.0
1400.0
1500.0
1600.0
2000.0
smoke
是
否
emaciation
0.0
1.0
2.0
drinking
是
否
PTCD_ERCP
0.0
1.0
2.0
3.0
biliary_disease
是
否
gender
男
女
LC
是
否
target_3Y_death
是
否
surgery_result
1.0
2.0
3.0
4.0
AJCC_8
0.0
1.0
2.0
3.0
4.0
blood_type
0.0
1.0
2.0
3.0
MSKCC
1.0
2.0
3.0
HBcAb
是
否
surgery_plasm
0.0
200.0
400.0
600.0
800.0
1000.0
1200.0
2800.0
HBsAg
是
否
surgery_bleeding
20.0
50.0
100.0
150.0
200.0
300.0
350.0
400.0
450.0
500.0
600.0
700.0
800.0
1200.0
4000.0
smoke
是
否
emaciation
0.0
1.0
2.0
drinking
是
否
PTCD_ERCP
0.0
1.0
2.0
3.0
biliary_disease
是
否
gender
男
女
LC
是
否
target_3Y_death
是
否


In [557]:
# 连续变量数据统计
def stats_continuous(df_model,continuous_col):
    # 统计全变量体系各变量的平均数、上下四分位数、缺失率
    feature_quarter_list=[]
    mean_quarter_list=[]
    feature_std_list=[]
    mean_std_list=[]
    miss_list=[]
    for i in continuous_col:
        # 计算上下四分位、均值、标准差
        try:
            data = df_model[i].astype('float')
            stat_result = pd.DataFrame(data.describe())
            mean_value=stat_result.loc['mean',i]
            up_quarter=stat_result.loc['25%',i]
            down_quarter=stat_result.loc['75%',i]
            std_value=stat_result.loc['std',i]
        except:
            mean_value=np.nan
            up_quarter=np.nan
            down_quarter=np.nan
        # 计算缺失率
        if df_model[i].isnull().sum()==0:
            miss_rate='0%'
        else:
            miss_rate=df_model[i].isnull().sum()/df_model.shape[0]
            miss_rate="%.2f%%" % (miss_rate * 100)      # 百分数输出
        miss_list.append(miss_rate)
        # mean(quarter)
        feature_quarter_list.append(i+'，mean（IQR）')
        mean_quarter_list.append('%.2f(%.2f-%.2f)' % (mean_value,up_quarter,down_quarter))
        # mean(std)
        feature_std_list.append(i+'，mean±std')
        mean_std_list.append('%.2f±%.2f' % (mean_value,std_value))

    df_continuous_quarter=pd.DataFrame(data={'特征':feature_quarter_list,
                            'mean_quarter_list':mean_quarter_list,
                            'miss_list':miss_list})
    df_continuous_std=pd.DataFrame(data={'特征':feature_std_list,
                            'mean_std_list':mean_std_list,
                            'miss_list':miss_list})
    return df_continuous_quarter

In [558]:
df_continuous_stat_tran=stats_continuous(df_after_ML_tran,continuous_col)
df_continuous_stat_tran.to_excel(project_path+'/data/result/cox/df_1.7.2.4_after_连续_tran_数据统计.xlsx')
df_continuous_stat_test=stats_continuous(df_after_ML_test,continuous_col)
df_continuous_stat_test.to_excel(project_path+'/data/result/cox/df_1.7.2.4_after_连续_test_数据统计.xlsx')

#### 处理Ml变量

In [559]:
# 查看缺失率
for i in df_dataset_cox_after_importance.columns:
    percent_col= df_dataset_cox_after_importance[i].isnull().sum()/df_dataset_cox_after_importance.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: surgery_result 10.20%
每列缺失率: AJCC_8 6.87%
每列缺失率: blood_type 71.18%
每列缺失率: tumor_CA19-9 1.33%
每列缺失率: MSKCC 7.54%
每列缺失率: tumor_CA125 34.15%
每列缺失率: HBcAb 15.30%
每列缺失率: surgery_plasm 10.42%
每列缺失率: HBsAg 9.53%
每列缺失率: surgery_bleeding 9.76%
每列缺失率: tumor_size 22.84%
每列缺失率: smoke 0.00%
每列缺失率: emaciation 14.19%
每列缺失率: drinking 4.88%
每列缺失率: DB 12.20%
每列缺失率: tumor_AFP 3.99%
每列缺失率: tumor_CEA 2.44%
每列缺失率: PTCD_ERCP 5.76%
每列缺失率: biliary_disease 0.44%
每列缺失率: gender 0.22%
每列缺失率: LC 25.28%
每列缺失率: target_3Y_death 25.06%
每列缺失率: target_survival_month 25.06%


In [560]:
# 删除缺失超过20%的变量
df_after_ML_tran_cox=df_after_ML_tran.drop(['blood_type','tumor_CA125','tumor_size','LC'],axis=1)
df_after_ML_test_cox=df_after_ML_test.drop(['blood_type','tumor_CA125','tumor_size','LC'],axis=1)

In [561]:
df_after_ML_tran_cox.shape

(360, 19)

In [562]:
# 剔除缺失值
for i in df_after_ML_tran_cox.columns:
    df_after_ML_tran_cox = df_after_ML_tran_cox[df_after_ML_tran_cox[i].notnull()]
    df_after_ML_test_cox = df_after_ML_test_cox[df_after_ML_test_cox[i].notnull()]

In [563]:
df_after_ML_tran_cox.shape

(160, 19)

In [564]:
# 分类不平衡变量
for i in df_dataset_cox_after_importance.columns:
    if df_dataset_cox_after_importance[i].nunique() < 2:
        print(i)
        del df_after_ML_tran_cox[i]
        del df_after_ML_test_cox[i]
        continue
    if df_dataset_cox_after_importance[i].nunique() == 2:
        # 如果分类变量中某一变量的占比超过90%，则删除该指标
        num_1 = df_dataset_cox_after_importance[i].value_counts()  # df一列中不同变量的数目
        num_2 = num_1.div(df_dataset_cox_after_importance.shape[0])  # div除法，所有元素都除以相同数值
        num_3 = num_2.max()  # 取出最大值
        if num_3 >= 0.9:
            print(i)
            del df_after_ML_tran_cox[i]
            del df_after_ML_test_cox[i]

In [565]:
df_after_ML_tran_cox.columns

Index(['surgery_result', 'AJCC_8', 'tumor_CA19-9', 'MSKCC', 'HBcAb',
       'surgery_plasm', 'HBsAg', 'surgery_bleeding', 'smoke', 'emaciation',
       'drinking', 'DB', 'tumor_AFP', 'tumor_CEA', 'PTCD_ERCP',
       'biliary_disease', 'gender', 'target_3Y_death',
       'target_survival_month'],
      dtype='object')

In [566]:
df_after_ML_tran_cox.to_excel(project_path+'/data/result/cox/df_1.7.2.5_after_ML_tran_cox.xlsx')
df_after_ML_test_cox.to_excel(project_path+'/data/result/cox/df_1.7.2.5_after_ML_test_cox.xlsx')

#### 处理全部变量

In [577]:
# 查看缺失率
for i in df_dataset_cox_after.columns:
    percent_col= df_dataset_cox_after[i].isnull().sum()/df_dataset_cox_after.shape[0]
    percent_col="%.2f%%" % (percent_col * 100)
    print('每列缺失率:',i,percent_col)

每列缺失率: id 0.00%
每列缺失率: sampling 0.00%
每列缺失率: gender 0.22%
每列缺失率: age 0.22%
每列缺失率: height 31.93%
每列缺失率: weight 31.93%
每列缺失率: BMI 31.93%
每列缺失率: jaundice 0.00%
每列缺失率: emaciation 14.19%
每列缺失率: breath_disease 0.22%
每列缺失率: cardio_disease 0.22%
每列缺失率: nbdd 0.22%
每列缺失率: urinary_disease 0.67%
每列缺失率: endocrine_disease 0.44%
每列缺失率: biliary_disease 0.44%
每列缺失率: other_disease 0.89%
每列缺失率: smoke 0.00%
每列缺失率: drinking 4.88%
每列缺失率: family_history 1.55%
每列缺失率: blood_type 71.18%
每列缺失率: WBC 12.42%
每列缺失率: HGB 13.08%
每列缺失率: PLT 15.30%
每列缺失率: TB 1.11%
每列缺失率: DB 12.20%
每列缺失率: TP 16.63%
每列缺失率: ALB 3.77%
每列缺失率: LG 66.30%
每列缺失率: AG 32.59%
每列缺失率: PAB 23.06%
每列缺失率: ALT 11.97%
每列缺失率: AST 18.63%
每列缺失率: GT 28.60%
每列缺失率: ALP 30.60%
每列缺失率: tumor_AFP 3.99%
每列缺失率: tumor_CEA 2.44%
每列缺失率: tumor_CA19-9 1.33%
每列缺失率: tumor_CA125 34.15%
每列缺失率: TF 78.71%
每列缺失率: tumor_size 22.84%
每列缺失率: HBsAg 9.53%
每列缺失率: HBeAg 15.08%
每列缺失率: HBeAb 15.08%
每列缺失率: HBcAb 15.30%
每列缺失率: HCVAb 15.96%
每列缺失率: LC 25.28%
每列缺失率: AJCC_8 6.87%
每列缺失率: Gazzani

In [578]:
df_dataset_cox_after.shape

(451, 75)

In [579]:
# 删除无效变量
df_dataset_cox_after=df_dataset_cox_after.drop(['id','sampling','手术日期','target_1Y_death','target_death'],axis=1)
# 删除缺失超过20%的变量
df_dataset_cox_after=df_dataset_cox_after.drop(['height','weight','BMI','blood_type','LG','AG','PAB','GT','ALP','tumor_CA125',
                                                'TF','tumor_size','LC','surgery_CP','gene_MSI','TMB','IHC_cdx2','IHC_cea',
                                               'IHC_ck5','IHC_ck7','IHC_ck19','IHC_ck20','IHC_muc1','IHC_moc31','IHC_pd1',
                                               'IHC_pdl1'],axis=1)

In [580]:
seed_index

75

In [581]:
# 分类不平衡变量
for i in df_dataset_cox_after.columns:
    if df_dataset_cox_after[i].nunique() < 2:
        print(i)
        del df_dataset_cox_after[i]
        continue
    if df_dataset_cox_after[i].nunique() == 2:
        # 如果分类变量中某一变量的占比超过90%，则删除该指标
        num_1 = df_dataset_cox_after[i].value_counts()  # df一列中不同变量的数目
        num_2 = num_1.div(df_dataset_cox_after.shape[0])  # div除法，所有元素都除以相同数值
        num_3 = num_2.max()  # 取出最大值
        if num_3 >= 0.9:
            print(i)
            del df_dataset_cox_after[i]

breath_disease
urinary_disease
family_history


In [583]:
df_dataset_cox_after.columns

Index(['gender', 'age', 'jaundice', 'emaciation', 'cardio_disease', 'nbdd',
       'endocrine_disease', 'biliary_disease', 'other_disease', 'smoke',
       'drinking', 'WBC', 'HGB', 'PLT', 'TB', 'DB', 'TP', 'ALB', 'ALT', 'AST',
       'tumor_AFP', 'tumor_CEA', 'tumor_CA19-9', 'HBsAg', 'HBeAg', 'HBeAb',
       'HBcAb', 'HCVAb', 'AJCC_8', 'Gazzaniga_T', 'MSKCC', 'Blumgart_T',
       'Bismuth_C', 'PTCD_ERCP', 'surgery_bleeding', 'surgery_CRCS',
       'surgery_plasm', 'surgery_result', 'gene_mutation',
       'target_survival_month', 'target_3Y_death'],
      dtype='object')

In [584]:
# 全部数据集划分
df_after_all_tran_cox, df_after_all_test_cox = train_test_split(df_dataset_cox_after, test_size=0.2, random_state=seed_index)

In [585]:
# 剔除缺失值
for i in df_dataset_cox_after.columns:
    df_after_all_tran_cox = df_after_all_tran_cox[df_after_all_tran_cox[i].notnull()]
    df_after_all_test_cox = df_after_all_test_cox[df_after_all_test_cox[i].notnull()]

In [586]:
df_dataset_cox_after.shape

(451, 41)

In [587]:
df_after_all_tran_cox.to_excel(project_path+'/data/result/cox/df_1.7.2.6_after_all_tran_cox.xlsx')
df_after_all_test_cox.to_excel(project_path+'/data/result/cox/df_1.7.2.6_after_all_test_cox.xlsx')

#### R: ML + backward stepwise for postoperation

In [588]:
'''
手术结果	0.419
AJCC_8分期	0.760
肿瘤指标CA19-9	0.001
肿瘤指标CEA	0.053
'''

'\n手术结果\t0.419\nAJCC_8分期\t0.760\n肿瘤指标CA19-9\t0.001\n肿瘤指标CEA\t0.053\n'

#### R: 模型筛选能力对比，c_index和BS

In [589]:
'''
全部变量
全部变量+backward stepwise
机器学习变量
机器学习变量 + backward stepwise
'''

'\n全部变量\n全部变量+backward stepwise\n机器学习变量\n机器学习变量 + backward stepwise\n'

#### python: 计算c_index和BS

In [590]:
# #### 术后预后3年生存期

# # 删除1年生存期的空值
# df_dataset_cox_after_3Y=df_dataset_cox_after_importance[df_dataset_cox_after_importance.target_3Y_survival.notnull()]
# # 删除无用变量
# df_dataset_cox_after_3Y=df_dataset_cox_after_3Y.drop(['target_1Y_survival','target_1Y_death', 'target_5Y_death',
#                                             'LC','target_5Y_survival', 'target_8Y_death', 'target_8Y_survival'],axis=1)

# df_dataset_cox_after_3Y.shape

# df_dataset_cox_after_3Y.columns

# # 剔除缺失值
# for i in df_dataset_cox_after_3Y.columns:
#     df_dataset_cox_after_3Y = df_dataset_cox_after_3Y[df_dataset_cox_after_3Y[i].notnull()]

# df_dataset_cox_after_3Y.sampling.value_counts()

# df_dataset_cox_after_3Y.to_excel(project_path+'/data/result/cox/df_1.7.1_dataset_cox_after_3Y_ML.xlsx')

# # 计算c_index of ML
# from lifelines import CoxPHFitter
# from lifelines.utils import concordance_index

# cph = CoxPHFitter()
# cph.fit(df_dataset_cox_after_3Y, duration_col='target_3Y_survival',event_col='target_3Y_death',cluster_col='sampling',show_progress=True)
# c_index_3Y = cph.concordance_index_
# print(c_index_3Y)

# # 术后3年SPSS：backward再次筛选
# col_backward_3Y=['surgery_result','AJCC_8','tumor_CA199','biliary_disease']
# col_backward_3Y.insert(0,'sampling')
# col_backward_3Y.extend(['target_3Y_death','target_3Y_survival'])
# df_dataset_cox_after_3Y_backward=df_dataset_cox_after_3Y[col_backward_3Y]

# # 计算c_index of ML+SPSS
# from lifelines import CoxPHFitter
# from lifelines.utils import concordance_index

# cph = CoxPHFitter()
# cph.fit(df_dataset_cox_after_3Y_backward, duration_col='target_3Y_survival',event_col='target_3Y_death',cluster_col='sampling',show_progress=True)
# c_index_3Y_backward = cph.concordance_index_
# print(c_index_3Y_backward)

# # 计算3年生存期的BS
# from sksurv.metrics import brier_score
# from sksurv.datasets import load_gbsg2
# from sksurv.linear_model import CoxPHSurvivalAnalysis
# from sksurv.metrics import brier_score
# from sksurv.preprocessing import OneHotEncoder

# #### 术后预后3年生存期

# df_dataset_cox_after_3Y =pd.read_excel(project_path +'/data/result/cox/df_1.7.1_dataset_cox_after_3Y_ML.xlsx')
# if 'Unnamed: 0' in df_dataset_cox_after_3Y.columns:
#     df_dataset_cox_after_3Y = df_dataset_cox_after_3Y.drop(['Unnamed: 0'], axis=1)

# # 转换成boolean值
# df_dataset_cox_after_3Y['target_death_binary']=df_dataset_cox_after_3Y.target_3Y_death.astype(bool)
# # 构建BStrain_data和test_data
# bs_y = [(d,s) for d,s in zip(df_dataset_cox_after_3Y.target_death_binary,df_dataset_cox_after_3Y.target_3Y_survival)]
# bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

# df_dataset_cox_after_3Y.columns

# # 术后3年BS_after_3Y
# bs_x_after_3Y = df_dataset_cox_after_3Y.drop(['target_3Y_death','target_3Y_survival','sampling'],axis=1)
# est_after_3Y = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_after_3Y, bs_y)
# survs_after_3Y = est_after_3Y.predict_survival_function(bs_x_after_3Y)
# preds_after_3Y = [fn(10) for fn in survs_after_3Y]
# times, bs_after_3Y = brier_score(bs_y, bs_y, preds_after_3Y, 10)
# print(bs_after_3Y)

# # 术后3年SPSS：backward再次筛选
# bs_x_after_3Y_backward=bs_x_after_3Y[['surgery_result','AJCC_8','tumor_CA199','biliary_disease']]
# est_after_3Y_backward=CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_after_3Y_backward, bs_y)
# survs_after_3Y_backward = est_after_3Y_backward.predict_survival_function(bs_x_after_3Y_backward)
# preds_after_3Y_backward = [fn(10) for fn in survs_after_3Y_backward]
# times, bs_after_3Y_backward = brier_score(bs_y, bs_y, preds_after_3Y_backward, 10)
# print(bs_after_3Y_backward)

## 术后风险评分模型

### SPSS-cox回归分析

In [591]:
'''
ML+SPSS:backward筛选的变量计算风险评分：
1年生存期cox系数：
    surgery_result 0.734
    AJCC_8 0.879
    tumor_CA199 0.001
    smoke 0.607
    emaciation -0.756
3年生存期cox系数：
    surgery_result 0.486
    AJCC_8 0.561
    tumor_CA199 0.001
    biliary_disease 0.491
5年生存期cox系数
    surgery_result 0.574
    AJCC_8 0.486
    tumor_CA199 0.001
    biliary_disease 0.436
8年生存期cox系数
    surgery_result 0.606
    AJCC_8 0.482
    tumor_CA199 0.001
    biliary_disease 0.444
'''

'\nML+SPSS:backward筛选的变量计算风险评分：\n1年生存期cox系数：\n    surgery_result 0.734\n    AJCC_8 0.879\n    tumor_CA199 0.001\n    smoke 0.607\n    emaciation -0.756\n3年生存期cox系数：\n    surgery_result 0.486\n    AJCC_8 0.561\n    tumor_CA199 0.001\n    biliary_disease 0.491\n5年生存期cox系数\n    surgery_result 0.574\n    AJCC_8 0.486\n    tumor_CA199 0.001\n    biliary_disease 0.436\n8年生存期cox系数\n    surgery_result 0.606\n    AJCC_8 0.482\n    tumor_CA199 0.001\n    biliary_disease 0.444\n'

In [592]:
# # 1年生存期cox-LR回归
# col_cox_select_1Y=['surgery_result','AJCC_8','tumor_CA199','smoke','emaciation']
# # 3年生存期cox-LR回归
# col_cox_select_3Y=['surgery_result','AJCC_8','tumor_CA199','biliary_disease']
# # 5年生存期cox-LR回归
# col_cox_select_5Y=['surgery_result','AJCC_8','tumor_CA199','biliary_disease']
# # 8年生存期cox-LR回归
# col_cox_select_8Y=['surgery_result','AJCC_8','tumor_CA199','biliary_disease']
# # 合并风险模型变量
# col_cox_select=list(set(col_cox_select_1Y + col_cox_select_3Y + col_cox_select_5Y + col_cox_select_8Y))

In [593]:
col_cox_select

['surgery_result',
 'tumor_CA19-9',
 'tumor_CEA',
 'surgery_plasm',
 'biliary_disease',
 'PLT',
 'AJCC_8',
 'emaciation',
 'smoke']

### compute risk-score

In [645]:
df_model_risk_after =pd.read_excel(project_path +'/data/processed_data/df_3.1.1_术后预后cox数据集.xlsx')
if 'Unnamed: 0' in df_model_risk_after.columns:
    df_model_risk_after = df_model_risk_after.drop(['Unnamed: 0'], axis=1)

In [646]:
df_model_risk_after.shape

(451, 75)

In [647]:
import math
# 计算风险评分
df_model_risk_after=df_model_risk_after.reset_index(drop=True)
# 缺失项补充为0
col_cox_select=['surgery_result','tumor_CA19-9','tumor_CEA','surgery_plasm','biliary_disease','PLT','AJCC_8','emaciation','smoke']
df_model_risk_after[col_cox_select]=df_model_risk_after[col_cox_select].fillna(0)
for i in range(df_model_risk_after.shape[0]):
    surgery_result=float(df_model_risk_after.loc[i,'surgery_result'])
    ca199=float(df_model_risk_after.loc[i,'tumor_CA19-9'])
    ajcc=float(df_model_risk_after.loc[i,'AJCC_8'])
    biliary = float(df_model_risk_after.loc[i,'biliary_disease'])
    emaciation=float(df_model_risk_after.loc[i,'emaciation'])
    smoke=float(df_model_risk_after.loc[i,'smoke'])
    plt_value=float(df_model_risk_after.loc[i,'PLT'])
    surgery_plasm=float(df_model_risk_after.loc[i,'surgery_plasm'])
    cea=float(df_model_risk_after.loc[i,'tumor_CEA'])


#     # 术后1年生存期risk_score计算
#     df_model_risk_after.loc[i,'risk_score_1Y']=0.734*surgery_result+0.001*ca199+0.879*ajcc+0.607*smoke-0.756*emaciation
#     # 术后3年生存期risk_score计算
#     df_model_risk_after.loc[i,'risk_score_3Y']=0.486*surgery_result+0.001*ca199+0.561*ajcc+0.491*biliary
    # 术后5年生存期risk_score计算
    df_model_risk_after.loc[i,'risk_score_3Y']=0.419*surgery_result+0.76*ajcc+0.001*ca199+0.053*cea
#     # 术后8年生存期risk_score计算
#     df_model_risk_after.loc[i,'risk_score_8Y']=0.606*surgery_result+0.001*ca199+0.482*ajcc+0.444*biliary
    # 现有方法3年生存期risk_score计算
    if plt_value <=0:
        df_model_risk_after.loc[i,'risk_score_existing']=ajcc*0.58+0.59*surgery_result+0.0004*surgery_plasm
    else:
        df_model_risk_after.loc[i,'risk_score_existing']=math.log(plt_value)*0.002+ajcc*0.58+0.59*surgery_result+\
                                                    0.0004*surgery_plasm


In [648]:
df_model_risk_after.shape

(451, 77)

In [649]:
df_model_risk_after.risk_score_3Y.describe()

count    451.000000
mean       3.221222
std        1.611756
min        0.000000
25%        2.305450
50%        3.030600
75%        3.961335
max       20.344000
Name: risk_score_3Y, dtype: float64

In [650]:
df_model_risk_after.risk_score_existing.describe()

count    451.000000
mean       2.409457
std        1.106319
min        0.000000
25%        1.761200
50%        2.340665
75%        2.910647
max        5.011693
Name: risk_score_existing, dtype: float64

In [651]:
df_model_risk_after.to_excel(project_path+'/data/result/cox/df_1.8.2_risk_score_after.xlsx')

### 术后分期

In [663]:
'''
R语言：风险评分分布直方图
根据风险评分分布直方图和中位数、四分卫点划分病理分期
'''

'\nR语言：风险评分分布直方图\n根据风险评分分布直方图和中位数、四分卫点划分病理分期\n'

In [652]:
# 术后3年生存期风险评分分期
df_model_risk_after['staging_after_3Y']=df_model_risk_after.risk_score_3Y.apply(lambda x: 1 if x<=2 else
                                                                         2 if 2<x<=3 else
                                                                         3 if 3<x<=4 else
                                                                         4 if 4<x<=5 else
                                                                         5 if x>5 else np.nan)

In [653]:
# 现有方法3年生存期风险评分分期
df_model_risk_after['staging_after_existing']=df_model_risk_after.risk_score_existing.apply(lambda x: 1 if x<=2 else
                                                                         2 if 2<x<=3 else
                                                                         3 if 3<x<=4 else
                                                                         4 if 4<x<=5 else
                                                                         5 if x>5 else np.nan)

In [654]:
df_model_risk_after.staging_after_3Y.value_counts()

2    184
3    123
4     66
5     41
1     37
Name: staging_after_3Y, dtype: int64

In [655]:
df_model_risk_after.staging_after_existing.value_counts()

2    179
1    173
4     61
3     36
5      2
Name: staging_after_existing, dtype: int64

In [657]:
df_model_risk_after.to_excel(project_path+'/data/result/cox/df_1.8.3_术后分期.xlsx')

### 划分数据集

In [658]:
df_model_risk_after.shape

(451, 79)

In [659]:
seed_index

75

In [660]:
from sklearn.model_selection import train_test_split
# 划分训练集和测试集，比例为8:2

df_after_staging_tran, df_after_staging_test = train_test_split(df_model_risk_after, test_size=0.2, random_state=seed_index)

In [661]:
print(df_after_staging_tran.shape)
print(df_after_staging_test.shape)

(360, 79)
(91, 79)


In [615]:
df_after_staging_tran.to_excel(project_path+'/data/result/cox/df_1.8.3_术后分期_训练集.xlsx')
df_after_staging_test.to_excel(project_path+'/data/result/cox/df_1.8.3_术后分期_测试集.xlsx')

### R语言: 风险评分直方画图

### R语言：术后分期生存分析图

### R语言：术后分期比较

### python比较术后分期

In [523]:
df_model_risk_after.columns

Index(['id', 'sampling', 'gender', 'age', 'height', 'weight', 'BMI',
       'jaundice', 'emaciation', 'breath_disease', 'cardio_disease', 'nbdd',
       'urinary_disease', 'endocrine_disease', 'biliary_disease',
       'other_disease', 'smoke', 'drinking', 'family_history', 'blood_type',
       'WBC', 'HGB', 'PLT', 'TB', 'DB', 'TP', 'ALB', 'LG', 'AG', 'PAB', 'ALT',
       'AST', 'GT', 'ALP', 'tumor_AFP', 'tumor_CEA', 'tumor_CA19-9',
       'tumor_CA125', 'TF', 'tumor_size', 'HBsAg', 'HBeAg', 'HBeAb', 'HBcAb',
       'HCVAb', 'LC', 'AJCC_8', 'Gazzaniga_T', 'MSKCC', 'Blumgart_T',
       'Bismuth_C', 'PTCD_ERCP', '手术日期', 'surgery_bleeding', 'surgery_CRCS',
       'surgery_plasm', 'surgery_CP', 'surgery_result', 'gene_mutation',
       'gene_MSI', 'TMB', 'IHC_cdx2', 'IHC_cea', 'IHC_ck5', 'IHC_ck7',
       'IHC_ck19', 'IHC_ck20', 'IHC_muc1', 'IHC_moc31', 'IHC_pd1', 'IHC_pdl1',
       'target_death', 'target_survival_month', 'target_1Y_death',
       'target_3Y_death', 'risk_score_3Y', 'risk

#### C_index

In [524]:
# from lifelines import CoxPHFitter
# from lifelines.utils import concordance_index
# # 计算1年生存期分期的c_index_us
# df_C_index_us = df_model_risk_after[['staging_after_1Y','target_1Y_death','target_1Y_survival']]
# df_C_index_us = df_C_index_us[df_C_index_us.target_1Y_survival.notnull()]
# cph = CoxPHFitter()
# cph.fit(df_C_index_us, duration_col='target_1Y_survival', event_col='target_1Y_death', show_progress=True)
# C_index_us_1Y = cph.concordance_index_
# print(C_index_us_1Y)

# # 计算3年生存期分期的c_index_us
# df_C_index_us = df_model_risk_after[['staging_after_3Y','target_3Y_death','target_3Y_survival']]
# df_C_index_us = df_C_index_us[df_C_index_us.target_3Y_survival.notnull()]
# cph = CoxPHFitter()
# cph.fit(df_C_index_us, duration_col='target_3Y_survival', event_col='target_3Y_death', show_progress=True)
# C_index_us_3Y = cph.concordance_index_
# print(C_index_us_3Y)

# # 计算5年生存期分期的c_index_us
# df_C_index_us = df_model_risk_after[['staging_after_5Y','target_5Y_death','target_5Y_survival']]
# df_C_index_us = df_C_index_us[df_C_index_us.target_5Y_survival.notnull()]
# cph = CoxPHFitter()
# cph.fit(df_C_index_us, duration_col='target_5Y_survival', event_col='target_5Y_death', show_progress=True)
# C_index_us_5Y = cph.concordance_index_
# print(C_index_us_5Y)

# # 计算8年生存期分期的c_index_us
# df_C_index_us = df_model_risk_after[['staging_after_8Y','target_8Y_death','target_8Y_survival']]
# df_C_index_us = df_C_index_us[df_C_index_us.target_8Y_survival.notnull()]
# cph = CoxPHFitter()
# cph.fit(df_C_index_us, duration_col='target_8Y_survival', event_col='target_8Y_death', show_progress=True)
# C_index_us_8Y = cph.concordance_index_
# print(C_index_us_8Y)

### python: sksurv计算BS

In [525]:
from sksurv.metrics import brier_score
from sksurv.datasets import load_gbsg2
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import brier_score
from sksurv.preprocessing import OneHotEncoder

ModuleNotFoundError: No module named 'sksurv'

In [ ]:
df_model_bs=pd.read_excel(project_path+'/data/result/modeling/df_1.7.2_术后分期.xlsx')
if 'Unnamed: 0' in df_model_bs.columns:
    df_model_bs = df_model_bs.drop(['Unnamed: 0'], axis=1)

In [ ]:
df_model_bs.shape

In [ ]:
df_model_bs.columns

In [ ]:
# 删除分期缺失值
col_bs_select=['staging_after_1Y','staging_after_3Y','staging_after_5Y','staging_after_8Y','AJCC_8','Gazzaniga_T','MSKCC','Blumgart_T']
for i in col_bs_select:
    df_model_bs=df_model_bs[df_model_bs[i].notnull()]

In [ ]:
df_model_bs.shape

In [ ]:
df_model_bs[col_bs_select].isnull().any()

#### 3Y--BS

In [ ]:
# 术后预后3年生存期BS，生成布尔型True、False
df_model_bs_3Y=df_model_bs[df_model_bs.target_3Y_survival.notnull()]
df_model_bs_3Y['target_death_binary']=df_model_bs_3Y.target_3Y_death.astype(bool)
# 构建BStrain_data和test_data
bs_y = [(d,s) for d,s in zip(df_model_bs_3Y.target_death_binary,df_model_bs_3Y.target_3Y_survival)]
bs_y=np.array(bs_y,dtype=[('cens', '?'), ('time', '<f8')])

In [ ]:
# 术后3年BS_us
bs_x_us = df_model_bs_3Y[['staging_after_3Y']]
est_us = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_us, bs_y)
survs_us = est_us.predict_survival_function(bs_x_us)
preds_us = [fn(10) for fn in survs_us]
times, bs_3Y_us = brier_score(bs_y, bs_y, preds_us, 10)
print(bs_3Y_us)

In [ ]:
# 术后3年BS_ajcc
bs_x_ajcc=df_model_bs_3Y[['AJCC_8']]
est_ajcc = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_ajcc, bs_y)
survs_ajcc = est_ajcc.predict_survival_function(bs_x_ajcc)
preds_ajcc = [fn(10) for fn in survs_ajcc]
times, bs_3Y_ajcc = brier_score(bs_y, bs_y, preds_ajcc, 10)
print(bs_3Y_ajcc)

In [ ]:
# 术后3年BS_gazz
bs_x_gazz=df_model_bs_3Y[['Gazzaniga_T']]
est_gazz = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_gazz, bs_y)
survs_gazz = est_gazz.predict_survival_function(bs_x_gazz)
preds_gazz = [fn(10) for fn in survs_gazz]
times, bs_3Y_gazz = brier_score(bs_y, bs_y, preds_gazz, 10)
print(bs_3Y_gazz)

In [ ]:
# 术后3年BS_mskcc
bs_x_mskcc=df_model_bs_3Y[['MSKCC']]
est_mskcc = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_mskcc, bs_y)
survs_mskcc = est_mskcc.predict_survival_function(bs_x_mskcc)
preds_mskcc = [fn(10) for fn in survs_mskcc]
times, bs_3Y_mskcc = brier_score(bs_y, bs_y, preds_mskcc, 10)
print(bs_3Y_mskcc)

In [ ]:
# 术后3年BS_blumgart
bs_x_blumgart=df_model_bs_3Y[['Blumgart_T']]
est_blumgart = CoxPHSurvivalAnalysis(ties="efron").fit(bs_x_blumgart, bs_y)
survs_blumgart = est_blumgart.predict_survival_function(bs_x_blumgart)
preds_blumgart = [fn(10) for fn in survs_blumgart]
times, bs_3Y_blumgart = brier_score(bs_y, bs_y, preds_blumgart, 10)
print(bs_3Y_blumgart)

## 多分类模型

### 未插补模型

In [ ]:
from sklearn.metrics import r2_score,average_precision_score,precision_recall_curve
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,accuracy_score

import xgboost
# XGBoost模型
xgb_model=xgboost.XGBClassifier(max_depth=5,
                        learning_rate=0.018,
                        n_estimators=500,
                        min_child_weight=0.6,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=5,
                        subsample=0.8,
                        colsample_bytree=0.6,
                        nthread=4,
                        scale_pos_weight=1,
                        random_state=3)

xgb_model.fit(tran_x_sm,tran_y_sm)
xgb_predictions=xgb_model.predict(test_x)


import lightgbm
# LightGBM模型
lgbm_model=lightgbm.LGBMClassifier(iterations=300, 
                                  max_depth=8,
                                  min_child_weight=0.9,
                                  gamma=0.5,
                                   reg_lambda=5,
                                  subsample=0.4,
                                  learning_rate=0.2, 
                                  loss_function='CrossEntropy',
                                  random_state=3)
lgbm_model.fit(tran_x_sm,tran_y_sm)
lgbm_predictions=lgbm_model.predict(test_x)


import catboost
# CatBoost模型
cat_model=catboost.CatBoostClassifier(iterations=300, 
                                      learning_rate=0.2, 
                                      depth=6,
                                      l2_leaf_reg=2,
                                      loss_function='MultiClass',
                                      random_state=3)
cat_model.fit(tran_x_sm,tran_y_sm)
cat_predictions=cat_model.predict(test_x)

In [ ]:
test_y

In [ ]:
test_y

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
# 统一模型输出结果
df_model_result=pd.DataFrame(
    columns=['model','index','precision','recall','f1-score','support','accuracy','AUC','sensitivity','specificity'])

model_list=[xgb_model,lgbm_model,cat_model]
model_name_list=['XGBoost','LGBM','CatBoost']
for model,name in zip(model_list,model_name_list):
#     print(name)
    # 计算accuracy和AUC
    if name == 'TabNet':
        test_x=test_x.to_numpy()
    test_y_score=model.predict_proba(test_x)
    auc=roc_auc_score(test_y,test_y_score,average='micro',multi_class='ovr')
    auc=round(auc,4)
    accuracy=accuracy_score(test_y,model.predict(test_x))
    accuracy=round(accuracy,4)
    # 计算灵敏度sensitivity和特异度specificity
    # 计算灵敏度、特异度
#     tn, fp, fn, tp = confusion_matrix(test_y,model.predict(test_x),multi_class='ovr').rival()
#     sensitivity=round(tp/(tp+fn),4)
#     specificity=round(tn/(fp+tn),4)
    df_model_result.loc[df_model_result.shape[0],['model','accuracy','AUC']]=\
                                                              [name,accuracy,auc]
    # 并入二分类的P-R-f1
    # 提取classification_report结果
    report = classification_report(test_y, model.predict(test_x), output_dict=True)  # output_dict转化为字典类型
    df_report = pd.DataFrame(report).transpose()  # 转置
    df_report=df_report.apply(lambda x: round(x,4),axis=0)
    df_report=df_report.reset_index(drop=True)
    df_model_result=pd.concat([df_model_result,df_report.loc[0:5,:].reset_index()],axis=0)
    df_model_result=df_model_result.reset_index(drop=True)

In [ ]:
df_model_result

In [ ]:
df_model_result.rename(columns={'model':'',
                               'index':'label'},inplace=True)
# 保存模型测试效果
df_model_result.to_excel(project_path+'/data/df_分类_模型测试效果_未插补.xlsx')

### 插补模型

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import r2_score,average_precision_score,precision_recall_curve
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,accuracy_score

import xgboost
# XGBoost模型
xgb_model=xgboost.XGBClassifier(max_depth=5,
                        learning_rate=0.018,
                        n_estimators=500,
                        min_child_weight=0.6,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=5,
                        subsample=0.8,
                        colsample_bytree=0.6,
                        nthread=4,
                        scale_pos_weight=1,
                        random_state=3)

xgb_model.fit(tran_x_sm,tran_y_sm)
xgb_predictions=xgb_model.predict(test_x)


import lightgbm
# LightGBM模型
lgbm_model=lightgbm.LGBMClassifier(iterations=300, 
                                  max_depth=8,
                                  min_child_weight=0.9,
                                  gamma=0.5,
                                   reg_lambda=5,
                                  subsample=0.4,
                                  learning_rate=0.2, 
                                  loss_function='CrossEntropy',
                                  random_state=3)
lgbm_model.fit(tran_x_sm,tran_y_sm)
lgbm_predictions=lgbm_model.predict(test_x)


import catboost
# CatBoost模型
cat_model=catboost.CatBoostClassifier(iterations=300, 
                                      learning_rate=0.2, 
                                      depth=6,
                                      l2_leaf_reg=2,
                                      loss_function='MultiClass',
                                      random_state=3)
cat_model.fit(tran_x_sm,tran_y_sm)
cat_predictions=cat_model.predict(test_x)


# 随机森林
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
# 列出参数列表
tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
# 进行参数的搜索组合
grid = GridSearchCV(RandomForestClassifier(), param_grid=tree_grid_parameter, cv=3)
# 根据已有数据去拟合随机森林模型
grid.fit(tran_x_sm, tran_y_sm)
rf_model = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'],
                            max_depth=8,
                            random_state=3)
rf_model.fit(tran_x_sm, tran_y_sm)
# 预测缺失值
rf_predictions = rf_model.predict(test_x)


# GBDT
# 列出参数列表
gbdt_model = GradientBoostingClassifier(n_estimators=300,
                            learning_rate=0.1,
                            max_depth=8,
                            subsample=0.4,
                            random_state=3)
gbdt_model.fit(tran_x_sm,tran_y_sm)
# 预测缺失值
gbdt_predictions = gbdt_model.predict(test_x)


# SVR
from sklearn.svm import SVR,SVC
# 回归模型
# svr = SVR(kernel='linear', C=1.25)
# 分类模型
svr_model = SVC(kernel='rbf',
          C=50,
          cache_size=200,
            probability=True,
          random_state=3)
svr_model.fit(tran_x_sm,tran_y_sm)
svr_predictions=svr_model.predict(test_x)


# Linear回归，Lasso回归，领回归，logistic回归
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet,LogisticRegression
lcv_model = LogisticRegression(penalty='l2',
                         C=5,
                        solver='lbfgs',
                         max_iter=100,
                        random_state=3)
# lcv = Lasso()
# lcv = Ridge()
lcv_model.fit(tran_x_sm, tran_y_sm)
lcv_predictions = lcv_model.predict(test_x)

# ANN
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

ANN_model = MLPClassifier(alpha=0.1, 
                    hidden_layer_sizes=[100,], 
                    solver='adam', 
                    activation='relu', 
                    random_state=3)
ANN_model.fit(tran_x_sm, tran_y_sm)
ANN_predictions=ANN_model.predict(test_x)


# TabNet
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
TabNet_model = TabNetMultiTaskClassifier(n_d=8, 
                               n_a=8,
                               n_steps=3, # Number of steps in the architecture (usually between 3 and 10)
                               gamma=1.5,
                               n_independent=2)  #TabNetRegressor()
tran_x_x, tran_x_valid, tran_y_y, tran_y_valid = train_test_split(tran_x_sm, tran_y_sm, test_size=0.125, random_state=3)

TabNet_model.fit(X_train=tran_x_x.to_numpy(), 
        y_train=tran_y_y.to_numpy().reshape(-1,1),
        max_epochs=200, 
        patience=20,
        batch_size=1024, 
        virtual_batch_size=128,
        num_workers=0,
        drop_last=False,
        loss_fn=[torch.nn.functional.cross_entropy]) # Optional, just an example of list usage

TabNet_predictions=TabNet_model.predict(test_x.to_numpy())

In [ ]:
# 统一模型输出结果
df_model_result=pd.DataFrame(
    columns=['model','index','precision','recall','f1-score','support','accuracy','AUC','sensitivity','specificity'])

model_list=[xgb_model,lgbm_model,cat_model,rf_model,gbdt_model,svr_model,lcv_model,ANN_model,TabNet_model]
model_name_list=['XGBoost','LGBM','CatBoost','RF','GBDT','SVR','LR','ANN','TabNet']
for model,name in zip(model_list,model_name_list):
#     print(name)
    if name == 'TabNet':
        test_x=test_x.to_numpy()
    test_y_score=model.predict_proba(test_x)
    auc=roc_auc_score(test_y,test_y_score,multi_class='ovr')
    auc=round(auc,4)
    accuracy=accuracy_score(test_y,model.predict(test_x))
    accuracy=round(accuracy,4)
    # 计算precision、recall、F1
    precision=precision_score(test_y,model.predict(test_x),average='macro')
    recall=recall_score(test_y,model.predict(test_x),average='macro')
    f1=f1_score(test_y,model.predict(test_x),average='macro')
    # 计算accuracy和AUC
#     if name == 'TabNet':
#         test_x=test_x.to_numpy()
#     y_one_hot = label_binarize(test_y, np.arange(6))
#     test_y_score=model.predict_proba(test_x)
#     auc=roc_auc_score(y_one_hot,test_y_score,average='micro')
#     auc=round(auc,4)

#     accuracy=accuracy_score(test_y,model.predict(test_x),average='macro')
#     accuracy=round(accuracy,2)
#     # 计算灵敏度sensitivity和特异度specificity
#     # 计算灵敏度、特异度、假阴性率、假阳性率
#     tn, fp, fn, tp = confusion_matrix(test_y,model.predict(test_x)).ravel()
#     sensitivity=round(tp/(tp+fn),2)
#     specificity=round(tn/(fp+tn),2)
#     FPR=round(fp/(fp+tn),2)
#     FNR=round(fn/(fn+tp),2)
#     # 计算约登指数
#     youden_index=sensitivity+specificity-1
    df_model_result.loc[df_model_result.shape[0],['model','accuracy','AUC',]]=\
                                            [name,accuracy,auc]
    # 并入二分类的P-R-f1
    # 提取classification_report结果
    report = classification_report(test_y, model.predict(test_x), output_dict=True)  # output_dict转化为字典类型
    df_report = pd.DataFrame(report).transpose()  # 转置
    df_report=df_report.apply(lambda x: round(x,2),axis=0)
    df_report=df_report.reset_index(drop=True)
    df_model_result=pd.concat([df_model_result,df_report.loc[0:1,:].reset_index()],axis=0)
    df_model_result=df_model_result.reset_index(drop=True)

In [ ]:
df_model_result

In [ ]:
df_model_result.rename(columns={'model':'',
                               'index':'label'},inplace=True)
# 保存模型测试效果
df_model_result.to_excel(project_path+'/data/df_分类_模型测试效果_插补.xlsx')

## 训练模型

### xgboost

In [ ]:
?precision_score

In [ ]:
# 直接使用xgboost和catboost包，而不是auto_ml
from sklearn.metrics import r2_score,precision_score,recall_score,f1_score,auc,accuracy_score
from sklearn.metrics import classification_report,confusion_matrix,average_precision_score,roc_curve,precision_recall_curve

import xgboost
# XGBoost模型
xgb_model=xgboost.XGBClassifier(max_depth=3,
                        learning_rate=0.009,
                        n_estimators=500,
                        min_child_weight=0.4,
                        eta=0.1,
                        gamma=0.4,
                        reg_lambda=10,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        nthread=4,
                        scale_pos_weight=1,
                        random_state=3)
xgb_model.fit(tran_x_sm,tran_y_sm)
xgb_predictions=xgb_model.predict(test_x)

xgb_precision = precision_score(test_y,xgb_predictions,average='macro')
xgb_recall=recall_score(test_y,xgb_predictions,average='macro')
xgb_f1=f1_score(test_y,xgb_predictions,average='macro')

# test_y_score=model.predict_proba(test_x)[:-1]
# xgb_auc=roc_auc_score(test_y,test_y_score,multi_class='ovr')
# xgb_auc=round(xgb_auc,2)

xgb_accuracy=accuracy_score(test_y,xgb_predictions)
print(xgb_precision,xgb_recall,xgb_f1,xgb_auc,xgb_accuracy)

# xgb_fpr, xgb_tpr, thresholds = roc_curve(test_y, test_y_score, pos_label=1)  #pos_label=2，表示值为2的实际值为正样本
# xgb_auc=auc(xgb_fpr,xgb_tpr)

cm2_LogR_model = confusion_matrix(test_y, xgb_model.predict(test_x))
print(cm2_LogR_model) #混肴矩阵
print(classification_report(test_y, xgb_model.predict(test_x)))

# # 计算灵敏度、特异度
# xgb_tn, xgb_fp, xgb_fn, xgb_tp = confusion_matrix(test_y,xgb_model.predict(test_x)).ravel()
# xgb_sensitivity=round(xgb_tp/(xgb_tp+xgb_fn),3)
# xgb_specificity=round(xgb_tn/(xgb_fp+xgb_tn),3)
# print(xgb_tn, xgb_fp, xgb_fn, xgb_tp)
# print(xgb_sensitivity,xgb_specificity)
# 提取classification_report结果
# 令output_dict转化为字典类型
report = classification_report(test_y, xgb_model.predict(test_x), output_dict=True)
df_report = pd.DataFrame(report).transpose()  # 转置
df_report = df_report.reset_index(drop=True)
print(df_report)
# xgb_ap = average_precision_score(test_y, predictions)
# xgb_precision, xgb_recall, _ = precision_recall_curve(test_y, predictions)
# # print(predictions)
# print(classification_report(test_y, xgb_model.predict(test_x)))

In [ ]:
df_model_result=pd.DataFrame(columns=['model','index','precision','recall','f1-score','support','accuracy','AUC','sensitivity','specificity'])

In [ ]:
df_model_result.loc[df_model_result.shape[0],['model','accuracy','AUC','sensitivity','specificity']]=\
                                                                    ['xgb',xgb_accuracy,xgb_auc,xgb_sensitivity,xgb_specificity]

In [ ]:
df_model_result

In [ ]:
df_model_result=pd.concat([df_model_result,df_report.loc[0:1,:].reset_index()],axis=0)

In [ ]:
df_model_result

In [ ]:
df_model_result.rename(columns={'model':'',
                               'index':''},inplace=True)

In [ ]:
df_model_result

In [ ]:
test_y.value_counts()

### LGBM

In [ ]:
?lgbm_model

In [ ]:
import lightgbm
# LightGBM模型
lgbm_model=lightgbm.LGBMClassifier(iterations=300, 
                                  max_depth=8,
                                  min_child_weight=0.9,
                                  gamma=0.5,
                                   reg_lambda=5,
                                  subsample=0.4,
                                  learning_rate=0.2, 
                                  loss_function='CrossEntropy',
                                  random_state=3)
lgbm_model.fit(tran_x_sm,tran_y_sm)
lgbm_predictions=lgbm_model.predict(test_x)

lgbm_precision = precision_score(test_y,lgbm_predictions)
lgbm_recall=recall_score(test_y,lgbm_predictions)
lgbm_f1=f1_score(test_y,lgbm_predictions)

test_y_score=lgbm_model.predict_proba(test_x)[:,-1]
lgbm_auc=roc_auc_score(test_y,test_y_score)
lgbm_accuracy=accuracy_score(test_y,lgbm_predictions)
print('lgbm',lgbm_precision,lgbm_recall,lgbm_f1,lgbm_auc,lgbm_accuracy)

### CatBoost

In [ ]:
?catboost.CatBoostClassifier

In [ ]:
import catboost
# CatBoost模型
cat_model=catboost.CatBoostClassifier(iterations=300, 
                                      learning_rate=0.2, 
                                      depth=6,
                                      l2_leaf_reg=2,
                                      subsample=1,
                                      loss_function='CrossEntropy',
                                      random_state=3)

cat_model.fit(tran_x_sm,tran_y_sm)
cat_predictions=cat_model.predict(test_x)

cat_precision = precision_score(test_y,cat_predictions)
cat_recall=recall_score(test_y,cat_predictions)
cat_f1=f1_score(test_y,cat_predictions)

test_y_score=cat_model.predict_proba(test_x)[:,-1]
cat_auc=roc_auc_score(test_y,test_y_score)
cat_accuracy=accuracy_score(test_y,cat_predictions)
print('catboost',cat_precision,cat_recall,cat_f1,cat_auc,cat_accuracy)

In [ ]:
print('catboost',cat_precision,cat_recall,cat_f1,cat_auc,cat_accuracy)

### Random Forest

In [ ]:
# 随机森林，GBDT
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
# 列出参数列表
tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
# 进行参数的搜索组合
grid = GridSearchCV(RandomForestClassifier(), param_grid=tree_grid_parameter, cv=3)
# 根据已有数据去拟合随机森林模型
grid.fit(tran_x_sm, tran_y_sm)
rf_model = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'],
                            max_depth=8,
                            random_state=3)
rf_model.fit(tran_x_sm, tran_y_sm)


# 预测缺失值
rf_predictions = rf_model.predict(test_x)

rf_precision = precision_score(test_y,rf_predictions)
rf_recall=recall_score(test_y,rf_predictions)
rf_f1=f1_score(test_y,rf_predictions)

test_y_score=rf_model.predict_proba(test_x)[:,-1]
rf_auc=roc_auc_score(test_y,test_y_score)
rf_accuracy=accuracy_score(test_y,rf_predictions)
print('rf',rf_precision,rf_recall,rf_f1,rf_auc,rf_accuracy)

### GBDT

In [ ]:
# GBDT
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
# 列出参数列表
tree_grid_parameter = {'n_estimators': [200,300,500],
                      'learning_rate': [0.01,0.1],
                      'max_depth': [3,5],
                      'subsample':[0.5,0.6,0.8]}
# 进行参数的搜索组合
grid = GridSearchCV(GradientBoostingClassifier(), param_grid=tree_grid_parameter, cv=3)
# 根据已有数据去拟合随机森林模型
grid.fit(tran_x_sm, tran_y_sm)

In [ ]:
grid.best_params_

In [ ]:
gbdt_model = GradientBoostingClassifier(n_estimators=300,
                            learning_rate=0.1,
                            max_depth=8,
                            subsample=0.4,
                            random_state=3)
gbdt_model.fit(tran_x_sm, tran_y_sm)
# 预测缺失值
gbdt_predictions = gbdt_model.predict(test_x)

gbdt_precision = precision_score(test_y,gbdt_predictions)
gbdt_recall=recall_score(test_y,gbdt_predictions)
gbdt_f1=f1_score(test_y,gbdt_predictions)

test_y_score=gbdt_model.predict_proba(test_x)[:,-1]
gbdt_auc=roc_auc_score(test_y,test_y_score)
gbdt_accuracy=accuracy_score(test_y,gbdt_predictions)
print('gbdt',gbdt_precision,gbdt_recall,gbdt_f1,gbdt_auc,gbdt_accuracy)

### SVR

In [ ]:
?SVC

In [ ]:
# SVR
from sklearn.svm import SVR,SVC
# 回归模型
# svr = SVR(kernel='linear', C=1.25)
# 分类模型
svr_model = SVC(kernel='rbf',
          C=50,
          cache_size=200,
            probability=True,
          random_state=3)
svr_model.fit(tran_x_sm,tran_y_sm)
svr_predictions=svr_model.predict(test_x)

svr_precision = precision_score(test_y,svr_predictions)
svr_recall=recall_score(test_y,svr_predictions)
svr_f1=f1_score(test_y,svr_predictions)

test_y_score=svr_model.predict_proba(test_x)[:,-1]
svr_auc=roc_auc_score(test_y,test_y_score)
svr_accuracy=accuracy_score(test_y,svr_predictions)
print('SVC',svr_precision,svr_recall,svr_f1,svr_auc,svr_accuracy)

In [ ]:
?svr_model

### KNN

In [ ]:
# KNN训练
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(tran_x,tran_y)
predictions=knn.predict(test_x)

### linear model

In [ ]:
?LogisticRegression

In [ ]:
# Linear回归，Lasso回归，领回归
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet,LogisticRegression
lcv_model = LogisticRegression(penalty='l2',
                         C=5,
                        solver='lbfgs',
                         max_iter=100,
                        random_state=3)
# lcv = Lasso()
# lcv = Ridge()

lcv_model.fit(tran_x_sm, tran_y_sm)
lcv_predictions = lcv_model.predict(test_x)

lcv_precision = precision_score(test_y,lcv_predictions)
lcv_recall=recall_score(test_y,lcv_predictions)
lcv_f1=f1_score(test_y,lcv_predictions)

test_y_score=lcv_model.predict_proba(test_x)[:,-1]
lcv_auc=roc_auc_score(test_y,test_y_score)
lcv_accuracy=accuracy_score(test_y,lcv_predictions)
print('lr',lcv_precision,lcv_recall,lcv_f1,lcv_auc,lcv_accuracy)

In [ ]:
?LogisticRegression

### ANN

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

ann_params = {'alpha': [0.0001,0.001,0.01,0.1,0.8],
              "solver": ['adam', 'sgd', 'lbfgs'],
              'hidden_layer_sizes': [(100,), (50,), (20,), (10,)]}

ANN = MLPClassifier(random_state=3)
grid = GridSearchCV(ANN, 
                    ann_params, 
                    cv=3, 
                    scoring='f1'
                   )
grid.fit(tran_x_sm, tran_y_sm)
# print(predictions)

In [ ]:
grid.best_params_

In [ ]:
?MLPClassifier

In [ ]:
# ANN
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

ANN = MLPClassifier(alpha=0.1, 
                    hidden_layer_sizes=[100,], 
                    solver='adam', 
                    activation='relu', 
                    random_state=3)
ANN.fit(tran_x_sm, tran_y_sm)
ann_predictions=ANN.predict(test_x)

ann_precision = precision_score(test_y,ann_predictions)
ann_recall=recall_score(test_y,ann_predictions)
ann_f1=f1_score(test_y,ann_predictions)

test_y_score=ANN.predict_proba(test_x)[:,-1]
ann_auc=roc_auc_score(test_y,test_y_score)
ann_accuracy=accuracy_score(test_y,ann_predictions)
print('ann',ann_precision,ann_recall,ann_f1,ann_auc,ann_accuracy)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(test_y, predictions))

cm2_LogR_model = confusion_matrix(test_y, predictions)
print(cm2_LogR_model) #混肴矩阵

### TabNet

In [ ]:
import ctypes
os.chdir('D:\Anaconda3\Lib\site-packages\~-rch\lib')
ctypes.cdll.LoadLibrary('caffe2_nvrtc.dll')

In [ ]:
?TabNetClassifier

In [ ]:
# TabNet
import torch
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
TabNet_model = TabNetMultiTaskClassifier(
                       cat_emb_dim=1,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2),
                       scheduler_params={"step_size":50, # how to use learning rate scheduler
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='entmax') # "sparsemax"
tran_x_x, tran_x_valid, tran_y_y, tran_y_valid = train_test_split(tran_x_sm, tran_y_sm, test_size=0.125, random_state=3)

TabNet_model.fit(X_train=tran_x_x, 
        y_train=tran_y_y.reshape(-1,1),
        max_epochs=200, 
        patience=20,
        batch_size=1024, 
        virtual_batch_size=128,
        num_workers=0,
        drop_last=False,
        loss_fn=[torch.nn.functional.cross_entropy]) # Optional, just an example of list usage

tabnet_predictions=TabNet_model.predict(test_x)

In [ ]:
test_y_score=model.predict_proba(test_x)
auc=roc_auc_score(test_y,test_y_score,multi_class='ovr')
auc=round(auc,4)
accuracy=accuracy_score(test_y,model.predict(test_x))
accuracy=round(accuracy,4)
# 计算precision、recall、F1
precision=precision_score(test_y,model.predict(test_x),average='macro')
recall=recall_score(test_y,model.predict(test_x),average='macro')
f1=f1_score(test_y,model.predict(test_x),average='macro')

In [ ]:
auc

In [ ]:
tabnet_predictions

In [ ]:
test_y_score

In [ ]:
test_y

In [ ]:
# TabNet
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
TabNet_model = TabNetClassifier(n_d=8, 
                               n_a=8,
                               n_steps=3, # Number of steps in the architecture (usually between 3 and 10)
                               gamma=1.5,
                               n_independent=2)  #TabNetRegressor()
tran_x_x, tran_x_valid, tran_y_y, tran_y_valid = train_test_split(tran_x_sm, tran_y_sm, test_size=0.125, random_state=3)

TabNet_model.fit(X_train=tran_x_x.to_numpy(), 
        y_train=tran_y_y.to_numpy(), 
        eval_set=[(tran_x_valid.to_numpy(), tran_y_valid.to_numpy())], 
        eval_name=['train'], 
        eval_metric=['auc'],
        max_epochs=100,
        patience=50,
        batch_size=128,
        virtual_batch_size=14,
        num_workers=0,
        drop_last=False)

tab_predictions=TabNet_model.predict(test_x.to_numpy())

tab_precision = precision_score(test_y,tab_predictions)
tab_recall=recall_score(test_y,tab_predictions)
tab_f1=f1_score(test_y,tab_predictions)

test_y_score = TabNet_model.predict_proba(test_x.to_numpy())[:,-1]
tab_auc = roc_auc_score(y_score=test_y_score, y_true=test_y)
tab_accuracy=accuracy_score(test_y,tab_predictions)
print('tab',tab_precision,tab_recall,tab_f1,tab_auc,tab_accuracy)

In [ ]:
print('tab',tab_precision,tab_recall,tab_f1,tab_auc,tab_accuracy)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
predictions=TabNet_model.predict(test_x.to_numpy())
print(classification_report(test_y, predictions))

cm2_LogR_model = confusion_matrix(test_y, predictions)
print(cm2_LogR_model) #混肴矩阵

## 计算评价指标

In [ ]:
# 计算R2和均方误差MSE
print('-----------------------计算R2和均方误差MSE---------------------------')

from sklearn.metrics import mean_squared_error  # 均方误差
from sklearn.metrics import mean_absolute_error  # 平方绝对误差
from sklearn.metrics import precision_score,recall_score,f1_score  # R square
# 调用

r2 = precision_score(test_y,xgb_predictions)
print('precision: ',r2)
mse=recall_score(test_y,xgb_predictions)
print('recall: ',mse)
mae=f1_score(test_y,xgb_predictions)
print('f1: ',mae)

In [ ]:
test_y.shape

In [ ]:
df_predictions= pd.DataFrame(data={'真实值':test_y,'预测值':xgb_predictions})
writer = pd.ExcelWriter(project_path + '/data/df_model_测试集结果.xlsx')
df_predictions.to_excel(writer)
writer.save()

## 画图

### 重要性评分

In [ ]:
?XGBClassifier()

In [ ]:
df_importance =pd.read_excel(project_path +'/data/result/modeling/df_1.6_模型重要性评分_after.xlsx')
if 'Unnamed: 0' in df_importance.columns:
    df_importance = df_importance.drop(['Unnamed: 0'], axis=1)

In [ ]:
df_importance.columns

In [ ]:
df_importance['重要性评分']=df_importance['重要性评分'].apply(lambda x: round(x,3))

In [ ]:
df_importance

In [ ]:
df_importance.特征.unique()

In [ ]:
df_importance['特征']=['手术结果','AJCC_8分期','血型','肿瘤指标CA19-9','MSKCC分期','肿瘤指标CA125','乙肝核心抗体','输血浆','乙肝表面抗原',
                     '术中出血','肿瘤最大径','吸烟史','有无消瘦','饮酒史','直接胆红素','肿瘤指标AFP','肿瘤指标CEA','术前减黄','有无胆道系统疾病',
                     '性别','有无肝硬化']

In [ ]:
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']  ##绘图显示中文
mpl.rcParams['axes.unicode_minus'] = False

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import rc

names = df_importance['特征']
index = np.arange(len(names))
plt.figure(figsize=(15, 8))
plt.bar(df_importance['特征'], df_importance['重要性评分'], width=0.7,
        color=(0.42941176470588235, 0.7294117647058823, 0.7490196078431373), tick_label=names)
# 设置坐标刻度值的大小
plt.tick_params(labelsize=15)
plt.xticks(rotation=60)

plt.ylabel('Importance Score',fontsize=20)
plt.xlabel('Variable Name',fontsize=20)
for a, b in zip(index, df_importance['重要性评分']):
    plt.text(a, b + 0.002, '%.4f' % b, ha='center', va='bottom', fontsize=10)
# plt.title('重要变量得分柱形图')
# plt.show()

### 散点图

In [ ]:
# 判断文件路径是否存在，如果不存在则创建该路径
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:  # 判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)  # makedirs 创建文件时如果路径不存在会创建这个路径

In [ ]:
# 画图
print('-----------------------画图---------------------------')
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']  ##绘图显示中文
mpl.rcParams['axes.unicode_minus'] = False

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import rc
rc('mathtext', default='regular')

# 散点图
# axis设置坐标轴的范围
# plt.axis([-20, 20, 0, 200])
# x为x轴中坐标x的值，y为y轴中坐标y的值，x与y都是长度相同的数组序列，color为点的颜色，marker为散点的形状，
# 折线图刻度调小，要不然点都堆到一块了
ax = plt.gca()
ax.set_xlim(0,10)
ax.set_ylim(0,10)
# plt.scatter(range(len(test_y)),test_y,c='r')
plt.scatter(test_y,predictions,c='b')
# 红色参照线
plt.plot(list(range(test_y.shape[0])), list(range(test_y.shape[0])),color='r')
# plt.plot(list(range(30)), list(range(30)),color='r')
plt.xlabel('Number of Events(unit)')
plt.ylabel('MTX Bone Suppression')

In [ ]:
# plt.show()
# 判断图片保存路径是否存在，否则创建
jpg_path = project_path + "/jpg"
mkdir(jpg_path)
plt.savefig(jpg_path + "/他克莫司血药浓度测试集散点图v2.0.jpg", dpi=300)
plt.clf()  # 删除前面所画的图

### AUC曲线

In [ ]:
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']  ##绘图显示中文
mpl.rcParams['axes.unicode_minus'] = False

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import rc

In [ ]:
plt.figure(figsize=(10,10))

# plt.plot(logistic_fpr, logistic_tpr,label='LogisticRegression(AUC = %0.2f)' % logistic_auc) 
# plt.plot(rf_fpr, rf_tpr,label='RandomForest(AUC = %0.2f)' % rf_auc) 
plt.plot(xgb_fpr, xgb_tpr,label='XGBoost(AUC = %0.2f)' % xgb_auc) 
# plt.plot(ann_fpr, ann_tpr,label='ANN(AUC = %0.2f)' % ann_auc) 

plt.plot([0, 1], [0, 1],linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('AUC Curve of MTX drug')
plt.legend(loc="lower right", fontsize=20)
plt.show()

### precision曲线

In [ ]:
plt.figure(figsize=(10,10))

plt.plot(logistic_recall, logistic_precision, label='LogisticRegression(AP = %0.2f)' % logistic_ap)
plt.plot(rf_recall, rf_precision,label='RandomForest(AP = %0.2f)' % rf_ap) 
plt.plot(xgb_recall, xgb_precision,label='XGBoost(AP = %0.2f)' % xgb_ap) 
plt.plot(ann_recall, ann_precision,label='ANN(AP = %0.2f)' % ann_ap) 
         
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 0.6])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend(loc="upper right", fontsize=20)
plt.show()

### SHAP图

#### summary_plot

In [ ]:
pd.value_counts(tran_y_sm)

In [ ]:
# SHAP图
from pylab import mpl
from matplotlib import pyplot as plt
mpl.rcParams['font.sans-serif'] = ['SimHei']  ##绘图显示中文
mpl.rcParams['axes.unicode_minus'] = False
from matplotlib import rc
rc('mathtext', default='regular')

import catboost,xgboost
import shap
shap.initjs()  # notebook环境下，加载用于可视化的JS代码
# CatBoost模型
cat_model=xgboost.XGBClassifier(max_depth=5,
                        learning_rate=0.001,
                        n_estimators=500,
                        min_child_weight=0.5,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=5,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        nthread=4,
                        scale_pos_weight=1,
                        random_state=3)
cat_model.fit(tran_x_sm, tran_y_sm)

explainer = shap.TreeExplainer(cat_model)
shap_values = explainer.shap_values(tran_x_sm)  # 传入特征矩阵X，计算SHAP值
# print(shap_values)

In [ ]:
tran_x_sm=tran_x_sm.rename(columns={'日剂量':'上一次日剂量',
                                   'gender':'性别',
                                   'age':'年龄',
                                   'test_result':'上一次TDM值'})

In [ ]:
?shap.summary_plot

In [ ]:
# summarize the effects of all the features
shap.summary_plot(shap_values, tran_x_sm,plot_size=(12,8),
                 class_names=['1mg','1.5mg','4mg','3mg','2.5mg','2mg'])

In [ ]:
tran_x_sm.columns

In [ ]:
type(shap_values)

In [ ]:
df_shap_values

In [ ]:
df_shap = pd.DataFrame(data={'features':shap_col,
                            'shap_pos':shap_pos_list,
                            'shap_neg':shap_neg_list})

In [ ]:
df_shap

In [ ]:
df_shap.to_excel(project_path+'/data/result/df_shap.xlsx')

In [ ]:
shap_list=[]
for i in range(df_shap_values.shape[1]-1):
    shap_value=df_shap_values.iloc[:,i].sum()
    shap_list.append(shap_value)
df_shap = pd.DataFrame(data={'features':tran_x_sm.columns,
                            'shap_value':shap_list})

In [ ]:
df_shap_values.iloc[:,i]

In [ ]:
shap_list

In [ ]:
df_shap

In [ ]:
writer = pd.ExcelWriter(project_path + '/data/result/df_shap值排序.xlsx')
df_shap.to_excel(writer)
writer.save()

#### multioutput_decision_plot

In [ ]:
?shap.multioutput_decision_plot

In [ ]:
col=df_model.columns.to_list()
col.remove('label')

In [ ]:
col

In [ ]:
shap.multioutput_decision_plot(shap_values,shap_values,row_index=3,
                              feature_names=col)

In [ ]:
shap_values

### 混淆矩阵图

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
import catboost
# CatBoost模型
cat_model=xgboost.XGBClassifier(max_depth=5,
                        learning_rate=0.001,
                        n_estimators=500,
                        min_child_weight=0.5,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=5,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        nthread=4,
                        scale_pos_weight=1,
                        random_state=3)

cat_model.fit(tran_x_sm,tran_y_sm)
cat_predictions=cat_model.predict(test_x)
# 计算混淆矩阵
cat_confusion=confusion_matrix(test_y,cat_predictions)

In [ ]:
cat_confusion

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(cat_confusion, cmap=plt.cm.Blues) # 在特定的窗口上显示图像
# 设置图表标题
plt.title('XGBoost Confusion Matrix',size=20)    # 图像标题
plt.colorbar()
# 设置坐标轴标题
font_x = {'family': 'Times New Roman',
         'weight': 'normal',
         'size': 20,}
plt.xlabel('prediction label',font_x)
plt.ylabel('True label',font_x)
# 设置坐标轴刻度
plt.tick_params(labelsize=23)  # 设置刻度值大小
label_names=['1g','1.5g','2mg','2.5mg','3mg','4mg']
plt.xticks(range(len(label_names)),label_names)
plt.yticks(range(len(label_names)),label_names)
# 显示数据
for first_index in range(len(cat_confusion)):    #第几行
    for second_index in range(len(cat_confusion[first_index])):    #第几列
        print(first_index, second_index)
        plt.text(second_index,first_index, cat_confusion[first_index][second_index],size=20)
plt.show()

In [ ]:
cat_confusion

In [ ]:
for first_index in range(len(cat_confusion)):    #第几行
    print(cat_confusion[first_index])
    break

In [ ]:
for second_index in range(len(cat_confusion[first_index])):    #第几列
    print(cat_confusion[first_index][second_index])

In [ ]:
# 显示数据
for first_index in range(len(cat_confusion)):    #第几行
    for second_index in range(len(cat_confusion[first_index])):    #第几列
        print(first_index, second_index)
        print(cat_confusion[first_index][second_index])
        plt.text(first_index, second_index, cat_confusion[first_index][second_index],size=20)
plt.show()

### tabnet mask graph

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
TabNet_model = TabNetClassifier()  #TabNetRegressor()
tran_x_x, tran_x_valid, tran_y_y, tran_y_valid = train_test_split(tran_x_sm, tran_y_sm, test_size=0.1, random_state=3)

TabNet_model.fit(X_train=tran_x_x.to_numpy(),  
        y_train=tran_y_y.to_numpy(), 
        eval_set=[(tran_x_valid.to_numpy(), tran_y_valid.to_numpy())], 
        eval_name=['train'], 
        eval_metric=['auc'],
        max_epochs=100,
        patience=15,
        batch_size=128,
        virtual_batch_size=15,
        num_workers=0,
        drop_last=False)

In [ ]:
explain_matrix,masks=TabNet_model.explain(tran_x_sm.to_numpy())

In [ ]:
from matplotlib import pyplot as plt
# fig = plt.figure(figsize=(40,40))
# ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
# plt.yticks(np.arange(0, len(explain_matrix), 1.0))
# plt.xticks(np.arange(0, len(explain_matrix[0]), 1.0))
# ax.set_xticklabels(tran_x_sm.columns, rotation=75)
# plt.ylabel('Sample Number')
# plt.xlabel('Variable')
# # plt.imshow(explain_matrix[:30])  # 显示30个

fig, axs = plt.subplots(1, 3, figsize=(20,20))
for i in range(3):
    axs[i].set_yticks(np.arange(0, len(explain_matrix), 1.0))  # 设置左边间距
    axs[i].set_xticks(np.arange(0, len(explain_matrix[0]), 2.0))
    axs[i].set_ylabel('Sample Number',size=20)
    axs[i].set_xlabel('Variable',size=20)
    # 设置坐标刻度值的大小
    axs[i].tick_params(labelsize=15)
    axs[i].imshow(masks[i][:30])
    axs[i].set_title(f"mask {i}")
    axs[i].set_xticklabels(tran_x_sm.columns[::2], rotation=90)

In [ ]:
tran_x_sm.columns